In [1]:
project_id = 'elife-data-pipeline'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'

In [2]:
import logging
import sys

import data_science_pipeline.configure_warnings

from data_science_pipeline.sql import get_sql
from data_science_pipeline.utils.bq import run_query_and_save_to_table, get_client

In [3]:
destination_table_name = ''.join([output_table_prefix, 'disambiguated_editor_papers'])

In [4]:
logging.basicConfig(level='INFO', stream=sys.stdout)

In [5]:
print('processing disambiguated_editor_papers')
run_query_and_save_to_table(
    client=get_client(project_id=project_id),
    query=get_sql('disambiguated_editor_papers.sql').format(
        project=project_id,
        dataset=output_dataset
    ),
    destination_dataset=output_dataset,
    destination_table_name=destination_table_name
)
print('done')

processing disambiguated_editor_papers
INFO:data_science_pipeline.utils.bq:ran query and saved to: de_dev.data_science_disambiguated_editor_papers, total rows: 621, took: 3.775s
done
